In [21]:
# Import Libraries

import pandas as pd
import altair as alt
import math
from PIL import Image
import os
import statistics
import re 

In [25]:
# Function definitions

def clean_df(df, param):
    dollars = []
    for col in df.columns:
        if isinstance(df[col][0], str) and df[col][0].startswith("$"):
            dollars.append(col)
    df[dollars] = df[dollars].replace('\s*\(\$\)', '', regex=True)
    df[dollars] = df[dollars].replace('[\$,]', '', regex=True).astype(float)
    
    truncrem = param['truncrem']
    if not math.isnan(truncrem) and truncrem > 0:
        trunc = param['trunc1']
        for i in range(len(df.columns)-1):
            df = df[df[df.columns[i+1]] > trunc]
    
    return df

def multiseries_line(df):
    new_df = {'x': [], 'y': [], 'color': []}
    cols = df.columns
    for index, row in df.iterrows():
        if index > 0:
            for i in range(len(cols)-1):
                if 'Unnamed' not in cols[i+1]:
                    new_df['x'].append(row[cols[0]])
                    new_df['y'].append(row[cols[i+1]])
                    new_df['color'].append(cols[i+1])
            
    return pd.DataFrame(new_df)
    

def clear_files():
    for domain in domains:
        d = domain[0].upper()
        for vtype in vtypes:
            t = vtype[0].upper()
            for i in range(5):
                for ext in exts:
                    fn = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)+ext+'.png'
                    if os.path.exists(fn):
                        os.remove(fn)
                        print(f"File '{fn}' deleted successfully.")

def show_files():
    for domain in domains:
        d = domain[0].upper()
        for vtype in vtypes:
            t = vtype[0].upper()
            for i in range(5):
                for ext in exts:
                    fn = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)+ext+'.png'
                    if os.path.exists(fn):
                        im = Image.open(fn)
                        im.show()

def minwidth(xs, vtype=''):
    uniques = xs.unique()
    #count = 0
    #sumx = 0
    maxx=0
    for x in uniques:
        #count = count + 1
        #sumx = sumx + len(str(x))
        length = len(str(x))
        if str(x).endswith('.'):
            length = length-1
        elif str(x).endswith('.0'):
            length = length-2
        if length > maxx:
            maxx = length
            
    total = len(uniques)
    #print(uniques)
    #print(count)
    #print(sumx)
    #print(maxx)
    #print(total)
    #avg = round(sumx/count)
    base = total * maxx * 8
    #if vtype == 'bar':
    #    base = base + 0.8
    #    if maxx > 4:
    #        base = base * 0.8
    #print(base)
    #print()
    if vtype != 'bar':
        base = max(base, 300)
    return base

def get_label_location(data, param):
    holder = []
    avg_holder = []
    a_holder = []
    grouper = data.groupby('color')
    for a, b in grouper:
        holder.append(b[b['x']==data['x'].iloc[-1]])
        avg_holder.append(statistics.mean(b['y']))
        a_holder.append(a)
    df2 = pd.concat(holder)
    newy = list(df2['y'])

    if len(newy) == 2:
        if max(newy) == min(newy):
            if avg_holder[0] > avg_holder[1]:
                newy[0] = newy[0] + param['y2']/40
                newy[1] = newy[1] - param['y2']/40
            else:
                newy[0] = newy[0] - param['y2']/40
                newy[1] = newy[1] + param['y2']/40

            df2['y'] = newy
        if max(df2['y']) - min(df2['y']) < (param['y2']/10):
            if newy[0] > newy[1]:
                newy[0] = newy[0] + param['y2']/50
                newy[1] = newy[1] - param['y2']/50
            else:
                newy[0] = newy[0] - param['y2']/50
                newy[1] = newy[1] + param['y2']/50

            df2['y'] = newy
    elif len(newy) == 3:
        if max(newy) - min(newy) < (param['y2']/10):
            if newy[0] == max(newy):
                newy[0] = newy[0] + param['y2']/30
                if avg_holder[1] == min(avg_holder):
                    newy[1] = newy[1] - param['y2']/30
                elif avg_holder[2] == min(avg_holder):
                    newy[2] = newy[2] - param['y2']/30
            elif newy[1] == max(newy):
                newy[1] = newy[1] + param['y2']/30
                if avg_holder[0] == min(avg_holder):
                    newy[0] = newy[0] - param['y2']/30
                elif avg_holder[2] == min(avg_holder):
                    newy[2] = newy[2] - param['y2']/30
            elif newy[2] == max(newy):
                newy[2] = newy[2] + param['y2']/30
                if avg_holder[0] == min(avg_holder):
                    newy[0] = newy[0] - param['y2']/30
                elif avg_holder[1] == min(avg_holder):
                    newy[1] = newy[1] - param['y2']/30

            df2['y'] = newy
        elif max(newy[0],newy[1]) - min(newy[0],newy[1]) < (param['y2']/10):
            if newy[0] > newy[1]:
                newy[0] = newy[0] + param['y2']/50
                newy[1] = newy[1] - param['y2']/50
            else:
                newy[0] = newy[0] - param['y2']/50
                newy[1] = newy[1] + param['y2']/50
        elif max(newy[0],newy[2]) - min(newy[0],newy[2]) < (param['y2']/10):
            if newy[0] > newy[2]:
                newy[0] = newy[0] + param['y2']/50
                newy[2] = newy[2] - param['y2']/50
            else:
                newy[0] = newy[0] - param['y2']/50
                newy[1] = newy[1] + param['y2']/50
        elif max(newy[2],newy[1]) - min(newy[2],newy[1]) < (param['y2']/10):
            if newy[2] > newy[1]:
                newy[2] = newy[2] + param['y2']/50
                newy[1] = newy[1] - param['y2']/50
            else:
                newy[2] = newy[2] - param['y2']/50
                newy[1] = newy[1] + param['y2']/50

            df2['y'] = newy
            
    return df2

def dualtitle(cols):
    y1 = re.sub("[\(\[].*?[\)\]]", "", cols[1]).strip()
    y2 = re.sub("[\(\[].*?[\)\]]", "", cols[2]).strip()
    return y1 + " vs. " + y2
    
    

In [31]:
# Set up global variables

show_img = False
save_img = True

parent = '../'
directory = parent + 'csv'
fn = directory+'/weather/bar/BW1.csv'
sample_data = pd.read_csv(fn)
fn = directory+'/weather/line/LW2.csv'
sample_data2 = pd.read_csv(fn)
sample_data2 = multiseries_line(sample_data2)

params = pd.read_csv(directory+'/parameters.csv')

domains = ['weather', 'election', 'disease', 'health', 'social', 'business', 'finance', 'physics', 'chemistry', 'lifescience']
#domains = ['physics', 'chemistry', 'lifescience']
vtypes = ['line', 'bar', 'sbubble', 'dual', 'histo']
exts = ['', '_base', '_dual', '_inverted', '_radius', '_inconsistent', '_trunc']

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']


In [464]:
### Example

sample_data['id'] = sample_data.index+1
sample_data['id'] = sample_data['id'].astype(str)
sample_data

alt.Chart(sample_data).mark_bar().encode(
    y = alt.Y('Month:O').sort(),
    x='Rainfall'
)

alt.Chart(...)

In [414]:
# Create a common chart object

chart = alt.Chart(sample_data2).mark_line().encode(
    x=alt.X('x:O').sort(),
    y=alt.Y('y:Q'),
    color=alt.Color('color', legend=None),
)

holder = []
grouper = sample_data2.groupby('color')
for a, b in grouper:
    holder.append(b[b['x']==sample_data2['x'].iloc[-1]])
    
df2 = pd.concat(holder)
df2

c2 = alt.Chart(df2).mark_point(filled=True).encode(
    x='x:O',
    y='y:Q',
    color='color'
)

c3 = c2.mark_text(dx=40).encode(
    text='color:N',
    color='color:N'
)


new = chart+c2+c3

new.configure_mark(
    opacity=1
)

alt.LayerChart(...)

In [312]:
### Base Bar Charts

for domain in domains:
    d = domain[0].upper()
    vtype = 'bar'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        
        chart = alt.Chart(data).mark_bar().encode(
            y=alt.Y(cols[0]+':O').title(param['xtitle']).sort(),
            x=alt.X(cols[1]+':Q', scale=alt.Scale(domain=[param['y1'],param['y2']])).title(param['ytitle'])
        ).properties(
            title=param['title']
            #width=minwidth(data['x'])
        )
                        
        if save_img:
            chart.save(saveto+'_base.png')
            
        if show_img:
            im = Image.open(saveto+'_base.png')
            im.show()
            
            

In [313]:
### Base Bar Charts - Vertical

for domain in domains:
    d = domain[0].upper()
    vtype = 'bar'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        
        chart = alt.Chart(data).mark_bar().encode(
            x=alt.X(cols[0]+':O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y(cols[1]+':Q', scale=alt.Scale(domain=[param['y1'],param['y2']])).title(param['ytitle'])
        ).properties(
            title=param['title'],
            width=minwidth(data[cols[0]], 'bar')
        )
                        
        if save_img:
            chart.save(saveto+'_basev.png')
            
        if show_img:
            im = Image.open(saveto+'_basev.png')
            im.show()

In [314]:
### Trunc Bar Charts

for domain in domains:
    d = domain[0].upper()
    vtype = 'bar'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        
        chart = alt.Chart(data).mark_bar(clip=True).encode(
            y=alt.Y(cols[0]+':O').sort().title(param['xtitle']),
            x=alt.X(cols[1]+':Q').scale(zero=False, domainMin=param['trunc1']).title(param['ytitle'])
        ).properties(
            title=param['title']
            #width=minwidth(data['x'])
        )
                        
        if save_img:
            chart.save(saveto+'_trunc.png')
            
        if show_img:
            im = Image.open(saveto+'_trunc.png')
            im.show()

In [485]:
### Base Line Charts - Label on Line

for domain in domains:
    d = domain[0].upper()
    vtype = 'line'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        data = multiseries_line(data)
                
        df2 = get_label_location(data, param)
            
        chart = alt.Chart(data).mark_line().encode(
            x=alt.X('x:O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y('y:Q', scale=alt.Scale(domain=[param['y1'],param['y2']])).title(param['ytitle']),
            color=alt.Color('color', legend=None),
        )        
            
        c2 = alt.Chart(df2).mark_point(filled=True, opacity=0).encode(
            x=alt.X('x:O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y='y:Q',
            color='color'
        )

        c3 = c2.mark_text(align='left',dx=15).encode(
            text='color:N',
            color='color:N'
        )


        chart = chart+c2+c3

        chart.configure_mark(
            opacity=0
        ).properties(
            title=param['title'],
            width=minwidth(data['x'])
        ).configure_view(
            stroke=None
        )

        if save_img:
            chart.save(saveto+'_base.png')

        if show_img:
            im = Image.open(saveto+'_base.png')
            im.show()
        

In [479]:
### Base Line Charts - Legend

for domain in domains:
    d = domain[0].upper()
    vtype = 'line'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        data = multiseries_line(data)
        
        #print(t+d+str(i+1))
        
        chart = alt.Chart(data).mark_line().encode(
            x=alt.X('x:O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y('y:Q', scale=alt.Scale(domain=[param['y1'],param['y2']])).title(param['ytitle']),
            color='color:N',
        ).properties(
            title=param['title'],
            width=minwidth(data['x'])
        ).configure_view(
            stroke=None
        )
        
        if save_img:
            chart.save(saveto+'_base.png')
            
        if show_img:
            im = Image.open(saveto+'_base.png')
            im.show()

In [482]:
### Inverted Line Charts - Label on Line

for domain in domains:
    d = domain[0].upper()
    vtype = 'line'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        data = multiseries_line(data)
                
        df2 = get_label_location(data, param)
            
        chart = alt.Chart(data).mark_line().encode(
            x=alt.X('x:O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y('y:Q', scale=alt.Scale(domain=[param['y2'],param['y1']])).title(param['ytitle']),
            color=alt.Color('color', legend=None),
        )        
            
        c2 = alt.Chart(df2).mark_point(filled=True, opacity=0).encode(
            x=alt.X('x:O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y='y:Q',
            color='color'
        )

        c3 = c2.mark_text(align='left',dx=15).encode(
            text='color:N',
            color='color:N'
        )


        chart = chart+c2+c3

        chart.configure_mark(
            opacity=0
        ).properties(
            title=param['title'],
            width=minwidth(data['x'])
        ).configure_view(
            stroke=None
        )

        if save_img:
            chart.save(saveto+'_base.png')

        if show_img:
            im = Image.open(saveto+'_base.png')
            im.show()

In [481]:
### Inverted Line Charts - Legend

for domain in domains:
    d = domain[0].upper()
    vtype = 'line'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        data = multiseries_line(data)
        
        if data['color'].nunique() > 3:
            print(t+d+str(i+1))
        
        chart = alt.Chart(data).mark_line().encode(
            x=alt.X('x:O').sort(),
            y=alt.Y('y:Q', scale=alt.Scale(domain=[param['y2'],param['y1']])),
            color='color:N',
        ).properties(
            title=param['title'],
            width=minwidth(data['x'])
        ).configure_view(
            stroke=None
        )
                        
        if save_img:
            chart.save(saveto+'_inverted.png')
            
        if show_img:
            im = Image.open(saveto+'_inverted.png')
            im.show()

In [334]:
### TO DO ###
# - PARAMS FOR NEW CHARTS!
# - Check data for new charts
#   - Check x-axis names (shorten)
# - Dual Axis (Line, w/ different scales, like %/$; base should create two side-by-side charts, dual axis should have both charts in dual axes)
#   - Which line is higher at given X? Correct answer is usually that they are different units.
#   - Does this chart show that Y1 causes Y2? ALT: Does this chart show that Y1 and Y2 are related? Correct answer is no.
# - Area Encoding (Bubble Chart)
#   - Ratios
# - Inconsistent Binning (Histogram)
#   - Compare above or below a certain x, or ranges within/around bins

In [14]:
## rename dirs

kinds = ['csv','img']
#for kind in kinds:
#    for domain in domains:
#        os.rename(os.path.join(parent, kind, domain, "area"), os.path.join(parent, kind, domain, "sbubble"))
#for kind in kinds:
#    for domain in domains:
#        for i in range(5):
#            os.rename(os.path.join(parent, kind, domain, "sbubble", "S"+domain[0].upper()+str(i+1)+"."+kind), os.path.join(parent, kind, domain, "sbubble", "S"+domain[0].upper()+str(i+1)+"."+kind))

In [43]:
#for domain in domains:
#    d = domain[0].upper()
#    vtype = 'histo'
#    t = vtype[0].upper()
#    for i in range(5):
#        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
#        data = pd.read_csv(fn)
#        cols = data.columns
#        #print(data)



domain = 'weather'
d = domain[0].upper()
vtype = 'histo'
t = vtype[0].upper()
i = 0
fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
data = pd.read_csv(fn)
cols = data.columns
alt.Chart(data).mark_bar().encode(
    alt.X(cols[0]+":Q", bin=True),
    y='count()',
)

alt.Chart(...)

In [60]:
data = pd.DataFrame({
    "bin_start": [8, 12, 14, 16, 18, 20, 22],
    "bin_end": [12, 14, 16, 18, 20, 22, 24],
    "count": [29, 71, 127, 94, 54, 17, 5]
})

alt.Chart(data).mark_bar().encode(
    x=alt.X("bin_start:Q", bin={"binned": True, "step": 4}),
    x2=alt.X2("bin_end:Q"),
    y='count:Q',
)

alt.Chart(...)

In [28]:
for domain in domains:
    d = domain[0].upper()
    vtype = 'dual'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        data = pd.read_csv(fn)
        cols = data.columns
        print(dualtitle(cols))
print()
        
for domain in domains:
    d = domain[0].upper()
    vtype = 'dual'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        data = pd.read_csv(fn)
        cols = data.columns
        print(cols[0])
print()

for domain in domains:
    d = domain[0].upper()
    vtype = 'dual'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        data = pd.read_csv(fn)
        cols = data.columns
        print(cols[1])
print()
        
for domain in domains:
    d = domain[0].upper()
    vtype = 'dual'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        data = pd.read_csv(fn)
        cols = data.columns
        print(cols[2])
print()
        
for domain in domains:
    d = domain[0].upper()
    vtype = 'dual'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        data = pd.read_csv(fn)
        cols = data.columns
        print(max(data[cols[1]]))
print()
        
for domain in domains:
    d = domain[0].upper()
    vtype = 'dual'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        data = pd.read_csv(fn)
        cols = data.columns
        print(max(data[cols[2]]))
        

Displacement vs. Velocity
Force vs. Acceleration
Kinetic Energy vs. Potential Energy
Voltage vs. Current
Mass vs. Weight
Temperature vs. Reaction Rate
Concentration vs. pH
Pressure vs. Volume
Temperature vs. Solubility
Concentration vs. Absorbance
Plant Height vs. Light Hours
Heart Rate vs. Activity Level
Temperature vs. Enzyme Activity
Population Size vs. Food Available
Glucose Level vs. Time

Time (s)
Time (s)
Height (m)
Time (s)
Object
Time (min)
Volume of Titrant (mL)
Time (s)
Compound
Sample ID
Day
Time (min)
Trial
Week
Minutes After Meal

Displacement (m)
Force (N)
Kinetic Energy (J)
Voltage (V)
Mass (kg)
Temperature (°C)
Concentration (mol/L)
Pressure (atm)
Temperature (°C)
Concentration (mol/L)
Plant Height (cm)
Heart Rate (bpm)
Temperature (°C)
Population Size
Glucose Level (mg/dL)

Velocity (m/s)
Acceleration (m/s²)
Potential Energy (J)
Current (A)
Weight (N)
Reaction Rate (mol/s)
pH
Volume (L)
Solubility (g/100 mL)
Absorbance (AU)
Light Hours
Activity Level (1-10)
Enzyme Act

In [15]:
### Base Dual Charts (side-by-side) - TODO!!!

for domain in domains:
    d = domain[0].upper()
    vtype = 'dual'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        print(param)
        
        data = clean_df(data, param)
        #data = multiseries_line(data)
        
        #print(t+d+str(i+1))
        
        chart1 = alt.Chart(data).mark_line().encode(
            x=alt.X(cols[0]+':O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y(cols[1]+':Q', scale=alt.Scale(domain=[param['y1'],param['y2']])).title(param['ytitle']),
            #color='color:N',
        ).properties(
            title=param['title'],
            width=minwidth(data[cols[0]])
        )

        chart2 = alt.Chart(data).mark_line().encode(
            x=alt.X(cols[0]+':O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y(cols[2]+':Q', scale=alt.Scale(domain=[param['y1'],param['dual2']])).title(param['dualtitle']),
            #color='color:N',
        ).properties(
            #title=param['title'],
            width=minwidth(data[cols[0]])
        )

        
        chart = alt.hconcat(chart1, chart2)
        if len(data[cols[0]]) > 15:
            chart = alt.vconcat(chart1, chart2)
        
        if save_img:
            chart.save(saveto+'_base.png')
            
        if show_img:
            im = Image.open(saveto+'_base.png')
            im.show()

{'id': 'DW1', 'vtype': 'D', 'domain': 'W', 'n': 1, 'title': 'Temperature vs. Humidity over Time', 'xtitle': 'Time', 'ytitle': 'Temperature (ºC)', 'dualtitle': 'Humidity (%)', 'y1': 0, 'y2': 35.0, 'trunc1': nan, 'truncrem': nan, 'dual2': 100.0}
{'id': 'DW2', 'vtype': 'D', 'domain': 'W', 'n': 2, 'title': 'Rainfall vs. Max Wind Speeds in June', 'xtitle': 'Date', 'ytitle': 'Rainfall (mm)', 'dualtitle': 'Wind Speed (km/h)', 'y1': 0, 'y2': 25.0, 'trunc1': nan, 'truncrem': nan, 'dual2': 25.0}
{'id': 'DW3', 'vtype': 'D', 'domain': 'W', 'n': 3, 'title': 'Sunshine Hours vs. Cloud Cover in July', 'xtitle': 'Date', 'ytitle': 'Sunshine Hours', 'dualtitle': 'Cloud Cover (%)', 'y1': 0, 'y2': 13.0, 'trunc1': nan, 'truncrem': nan, 'dual2': 100.0}
{'id': 'DW4', 'vtype': 'D', 'domain': 'W', 'n': 4, 'title': 'Pressure vs. Temperature over Time', 'xtitle': 'Time', 'ytitle': 'Pressure (hPa)', 'dualtitle': 'Temperature (°C)', 'y1': 0, 'y2': 1100.0, 'trunc1': nan, 'truncrem': nan, 'dual2': 30.0}
{'id': 'DW5',

ValidationError: nan is not of type 'string'

Failed validating 'type' in schema[0]:
    {'type': 'string'}

On instance:
    nan

In [486]:
### Dual Axis Charts - TODO!!!

for domain in domains:
    d = domain[0].upper()
    vtype = 'dual'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        #data = multiseries_line(data)
        
        #print(t+d+str(i+1))
        
        chart1 = alt.Chart(data).mark_line(color='#1F77B4').encode(
            x=alt.X(cols[0]+':O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y(cols[1]+':Q', scale=alt.Scale(domain=[param['y1'],param['y2']])).axis(title=param['ytitle'], titleColor='#1F77B4'),
            
        ).properties(
            title=param['title'],
            width=minwidth(data[cols[0]])
        )

        chart2 = alt.Chart(data).mark_line(color='#FF7F0E').encode(
            x=alt.X(cols[0]+':O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y(cols[2]+':Q', scale=alt.Scale(domain=[param['y1'],param['dual2']])).axis(title=param['dualtitle'], titleColor='#FF7F0E'),
            
        ).properties(
            #title=param['title'],
            width=minwidth(data[cols[0]])
        )

        chart = alt.layer(chart1, chart2).resolve_scale(
            y='independent'
        )
                        
        if save_img:
            chart.save(saveto+'_dual.png')
            
        if show_img:
            im = Image.open(saveto+'_dual.png')
            im.show()

In [ ]:
### Base Bubble Charts - TODO!!!

for domain in domains:
    d = domain[0].upper()
    vtype = 'area'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        data = multiseries_line(data)
        
        #print(t+d+str(i+1))
        
        chart = alt.Chart(data).mark_line().encode(
            x=alt.X('x:O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y('y:Q', axis=alt.title(param['ytitle']),
        ).properties(
            title=param['title'],
            width=minwidth(data['x'])
        )
                        
        if save_img:
            chart.save(saveto+'_base.png')
            
        if show_img:
            im = Image.open(saveto+'_base.png')
            im.show()

In [ ]:
### Radius Bubble Charts - TODO!!!

for domain in domains:
    d = domain[0].upper()
    vtype = 'area'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        data = multiseries_line(data)
        
        #print(t+d+str(i+1))
        
        chart = alt.Chart(data).mark_line().encode(
            x=alt.X('x:O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y('y:Q', scale=alt.Scale(domain=[param['y1'],param['y2']])).title(param['ytitle']),
            color='color:N',
        ).properties(
            title=param['title'],
            width=minwidth(data['x'])
        )
                        
        if save_img:
            chart.save(saveto+'_radius.png')
            
        if show_img:
            im = Image.open(saveto+'_radius.png')
            im.show()

In [ ]:
### Base Histograms - TODO

for domain in domains:
    d = domain[0].upper()
    vtype = 'histo'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        
        chart = alt.Chart(data).mark_bar().encode(
            x=alt.X(cols[0]+':O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y(cols[1]+':Q', scale=alt.Scale(domain=[param['y1'],param['y2']])).title(param['ytitle'])
        ).properties(
            title=param['title'],
            width=minwidth(data[cols[0]], 'bar')
        )
                        
        if save_img:
            chart.save(saveto+'_base.png')
            
        if show_img:
            im = Image.open(saveto+'_base.png')
            im.show()

In [ ]:
### Histograms Inconsistent Binning - TODO

for domain in domains:
    d = domain[0].upper()
    vtype = 'histo'
    t = vtype[0].upper()
    for i in range(5):
        fn = directory+'/'+domain+'/'+vtype+'/'+t+d+str(i+1)+'.csv'
        saveto = parent+'img/'+domain+'/'+vtype+'/'+t+d+str(i+1)
        data = pd.read_csv(fn)
        cols = [x for x in data.columns if 'Unnamed' not in x]
            
        param = params[params['id']==(t+d+str(i+1))].iloc[0].to_dict()

        data = clean_df(data, param)
        
        chart = alt.Chart(data).mark_bar().encode(
            x=alt.X(cols[0]+':O', axis=alt.Axis(labelAngle=0)).title(param['xtitle']).sort(),
            y=alt.Y(cols[1]+':Q', scale=alt.Scale(domain=[param['y1'],param['y2']])).title(param['ytitle'])
        ).properties(
            title=param['title'],
            width=minwidth(data[cols[0]], 'bar')
        )
                        
        if save_img:
            chart.save(saveto+'_inconsistent.png')
            
        if show_img:
            im = Image.open(saveto+'_inconsistent.png')
            im.show()